# Global Navigation Practice
---
<img src="resources/jderobot.png" width="13%" height="13%" style="float:left;padding-right:5px"/>
<img src="resources/holotaxi.png" width="35%" height="35%" style="float:right;padding:10px"/>


## 1 - Introduction
---

The objective of this practice is to implement the logic of a Gradient Path 
Planning (GPP) algorithm. Global navigation through GPP, consists of:

- Selected a destination, the GPP algorithm is responsible for finding the 
shortest path to it, avoiding, in the case of this practice, everything that is 
not road.
- Once the path has been selected, the logic necessary to follow this path and 
reach the objective must be implemented in the robot.

With this, it is possible for the robot to go to the marked destination 
autonomously and following the shortest path.


The solution can integrate one or more of the following levels
of difficulty, as well as any other one that occurs to you:
* Reach the goal.
* Optimize the way to find the shortest path.
* Arrive as quickly as possible to the destination.


For this practice a world has been designed for the Gazebo simulator (see section 2.1). The main task will consist of making the car reach the selected destiny.

To do so, the student needs to have at least the next knowledge:

* Python programming skills
* B driving license (well, that's a joke)
* GPP algorithm knowledge

##  2 - Exercise components
---
<img src="resources/globalnavigation_world.png" width="50%" height="500%" style="float:left;padding-right:10px;margin-top:15px;padding-left:10px;padding-bottom:7px;"/>

### 2.1 Gazebo Simulator

Gazebo simulator will be running in the background. The Gazebo world employed for this exercise has a 3D model of a City with its streets and obstacles (gray cubes). In a certain road, there is placed the robot. The disposal is the one shown in the image on the left:

As we said, the algorithm that you have to write will be used to control the holoTaxi robot (also present in the image, pointed by the cursor), which has a single pose sensor integrated that you can use.


### 2.2 Global Navigation Component

This component has been developed specifically to carry out this exercise. This component connects to Gazebo to communicate with the car robot (send orders to it and receive sensors data). The student has to modify this component and add code to accomplish the exercise. In particular, it is required to modify the execute method and the generatePath function as explained in 3.1 and 3.2.

## 3 - Exercise initialization
---
First of all, we need to connect with the robot through Gazebo simulator:

In [ ]:
import subprocess

simulator = subprocess.Popen(("gazebo", "cityLarge.world"))

Wait a few seconds until it has launched. Now we need to import all required python modules to carry out the practice, and then call ``GlobalNavigation`` class once. Run this code and wait a few seconds until GlobalNavigation initialization finishes with an ``OK`` message:

In [ ]:
#! usr/bin/env python
# -*- coding: utf-8 -*-

import math
import time
import numpy as np
from global_navigation import GlobalNavigation

gn = GlobalNavigation()

__NOTE__: Before continuing, you need to visit [the generatePath function](#path) section. Read the following points carefully (3.1 and 3.2)

### 3.1 - The execute() method

This method is where you have to write the code that manages the path generated by the generatePath function (see section 3.2) and transform that data into orders for the robot, so that it follows the marked path.
This path, once you have coded the function that gets it, will be available in this variable:
```
self.path
```
So you can use it in your execute() method if you needed it.
The execute() method from GlobalNavigation component will be called iteratively about 10 times per second. To understand how it works, we are going to print a message in each iteration so you can see the output. For that, run the following code:

In [ ]:
# Changing the execute method
def execute(self):
    print "Running execute iteration"
      
gn.setExecute(execute)
gn.play() # To play your execute method. Do not use it until you have finished generatePath

Now that you have seen it, stop printing this message. To do that, update the execute method, this time with an empty instruction:

In [ ]:
def execute(self):  
    pass
    
gn.setExecute(execute)
gn.stop() # To stop the execute method. Keep it stopped until you have coded the generatePath function (see 3.2)

### 3.2 - <a id='path'>The generatePath function</a>

This exercise uses the generatePath function to obtain the path generated by the GPP algorhythm after having selected the destination. The first task you must go through is to write the code that uses that GPP algorithm to get the shortest path. For that, you need to modify generatePath function as follows:

In [ ]:
def generatePath(self):
    print "GENERATING" # Add a message
      
    path = None
    return path # you need to return something in this function
gn.setGeneratePath(generatePath)
gn.generatePath() # To run generatePath function. Do not use it unless you are debugging. We will provide a generatePath button instead

### 3.3 - Running the code
---

Once everything is correctly initiallized, and both execute and generatePathe methods are done, run the following code to render the map where you must select the robot's destination. 

Run the next cell to get the map, so an image will appear in the output. Click on that image to select the robot's destination (you will see a blue mark where you have clicked). Once clicked, push the "Generate Path" button, so your generatePath() function will be called (__remember that you need to code it first!__). The resultant path (matrix size 400x400) of your function will be painted. 

__NOTE:__ Some changes made on the map widget, such as painting the path or painting the robot position, only will be seen if you place your cursor above the widget.

In [ ]:
%matplotlib notebook
import ipywidgets as widgets
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from IPython.core.display import HTML

def paintPath(p):
    for i in range(p.shape[0]):
        for j in range(p.shape[1]):
            if p[i][j] > 0:
                circle = plt.Circle((j, i), 1, color='green')
                ax.add_artist(circle)
                    
def calculatePath(b):
    path = gn.calcPath()
    gn.algorithm.path = path
    if path is not None:
        paintPath(path)
        
def onclick(event):
    
    ax.cla()
    ax.imshow(img)
    w.value = 'YOU HAVE CLICKED: x=%d, y=%d, xdata=%f, ydata=%f'%(
               event.x, event.y, event.xdata, event.ydata)

    circle = plt.Circle((event.xdata, event.ydata), 3, color='blue')
    ax.add_artist(circle)
    gn.coordinatesClicked = [event.xdata, event.ydata]
    gn.grid.setDestiny(int(round(event.xdata)), int(round(event.ydata)))
    robotpos = gn.algorithm.grid.worldToGrid(gn.sensor.getRobotX(), gn.sensor.getRobotY())
    robot = plt.Circle((robotpos[0], robotpos[1]), 3, color='yellow')
    ax.add_artist(robot)
    
img=mpimg.imread('resources/images/cityLargeBin.png', 'gray')

fig, ax = plt.subplots()
ax.imshow(img)
robotpos = gn.algorithm.grid.worldToGrid(gn.sensor.getRobotX(), gn.sensor.getRobotY())
robot = plt.Circle((robotpos[0], robotpos[1]), 3, color='yellow')
ax.add_artist(robot)
gn.setAx(ax)

w = widgets.HTML("Event information appears here when you click on the figure")

b = widgets.Button(
    description='Generate Path',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='gen',
    icon='legal'
)
    
cid = fig.canvas.mpl_connect('button_press_event', onclick)
b.on_click(calculatePath)
display(w)
display(b)
HTML("globalnavigation.html")

Once the path has been calculated, use the following instruction to run the execute() method:

In [ ]:
gn.play()

At this point, if you have coded correctly, you will see the car going to its destiny in the simulator.

### 3.4 - API:
---

- __grid's API__:
    - A grid class is provided to make this problem easier. That class have some methods that you may find useful:
    ```
    dest = self.grid.getDestiny() # gets the destiny selected on the map in 3.3
    pos = self.grid.getPose()     # gets the position on the map, (x, y)
    map = self.grid.getMap()      # gets the map of the image
    wCoords = self.grid.gridToWorld(pos[0], pos[1]) # Return the corresponding world coordinates of a grid pose
    imageCoords = self.grid.worldToGrid(wx, wy) # Return the corresponding image coordinates of world pose
    val = self.grid.getVal(i, j) # gets grid(i,j) value 
    self.grid.setVal(dest[0], dest[1], 0.0) # sets grid(i,j) value 
    self.grid.grid  # contains the grid object itself
    self.grid.setPathVal(posRobot[0], posRobot[1], valMin) # sets a value of the path
    self.grid.setPathFinded() # sets the path as found
    self.grid.getPath() # gets the final path. You need to return this in the generatePath fucntion
    ```
    we provide the code necessary to make relative (to the robot) your absolute coordinates. Use it as:
    ```
   relx, rely = self.absolutas2relativas(x, y, robotx, roboty, robotOrientation)
    ```

 - __robot's API__:
    
    - There is the way to get robot's position information:
    ```
    posRobotX = self.sensor.getRobotX() # x coordinate
    posRobotY = self.sensor.getRobotY() # y coordinate
    orientationRobot = self.sensor.getRobotTheta()  # orientation
    ```
    - And to send orders to it:
    ```
    self.vel.setW(angle) # sets angular speed
    self.vel.setV(speed) # sets linear speed
    ```

## 4 - Algorithm skeleton
---
We provide a __tentative basic__ skeleton where you can code your Global Navigation. This is one of the possible pipelines you can follow to get to the solution of this practice. Remember that you can follow the one that is easiest for you. We propose:

In [ ]:
def generatePath(self):
    
    if not self.initialized:
        # Initialize your variables if needed
        self.initiallized = True

    # mapIm is the image of the map
    mapIm = self.grid.getMap()
    # dest is the selected destination, and is a tuple (x, y)  
    dest = self.grid.getDestiny()
    if not dest:
        print("Destiny not yet selected!")
        return None
    # gridPos is the position of the selected on the map, (x, y)
    gridPos = self.grid.getPose()

    # Position of the robot
    world_robotX = self.sensor.getRobotX()
    world_robotY = self.sensor.getRobotY()
    posRobot = self.grid.worldToGrid(world_robotX, world_robotY)

   # GPP Algorithm

    # Evaluating the value of the field on position (dest[0], dest[1])

    # Expansion of the field
    
            # Do expansion of nodes
        
            # Cases of the margins, we check if we finish the expansion
           
            # We update the node that will expand values
           
   # Add Penalties' to Obstacles
                        
    # Add our grid and the penalty grid

    # Find the path
    
            #if found:
                self.grid.setPathFinded()
            
    print("Shortest path found!")
    return self.grid.getPath()

gn.setGeneratePath(generatePath)

In [ ]:
def execute(self):
    print("GOING TO DESTINATION")

    #EXAMPLE OF HOW TO SEND INFORMATION TO THE ROBOT ACTUATORS
    #self.vel.setV(10)
    #self.vel.setW(5)
    #self.vel.sendVelocities()

    # Position of the robot
    posRobotX = self.sensor.getRobotX()
    posRobotY = self.sensor.getRobotY()
    orientationRobot = self.sensor.getRobotTheta()

    # Destination
    dest = self.grid.getDestiny()
    destWorld = self.grid.gridToWorld(dest[0], dest[1])

    posRobotImage = self.grid.worldToGrid(posRobotX, posRobotY)

    # Find out if destination has been reached
    #if so:
        self.vel.setV(0)
        self.vel.setW(0)
        print("DESTINATION")
    #else:
        # We haven't arrived at the destination
        print(" NO DESTINATION")

        # calculate the next objective and the following to that one to do an interpolation
        
        # Interpolation
        targetInterpolationx = (target[0] + targetNext[0]) / 2
        targetInterpolationy = (target[1] + targetNext[1]) / 2            

        # Convert targetInterpolationx y targetInterpolationy to relative coordinates
        directionx,directiony = self.absolutas2relativas(targetInterpolationx,targetInterpolationy,posRobotX,posRobotY,orientationRobot)

        
        # calculate the angle between our position and the goal
        angle = math.atan((directiony/directionx))

        # Correct the position with that angle   

        # calculate angle of turn

        # set Angular speed
        if angle < 0:
            self.vel.setW(-angleTurn)
        else:
            self.vel.setW(angleTurn)

        # set Linear speed
        self.vel.setV(speed)
      
gn.setExecute(execute)